In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)

order_df = pd.read_csv("order.csv") 

account_df = pd.read_csv("account.csv")

account_df.head()

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30


In [90]:
order_df.shape

(6471, 6)

In [91]:
order_df['k_symbol'] = order_df['k_symbol'].fillna('Other')
order_df['k_symbol'].value_counts(dropna=False)
order_df.head(10)

,order_id,account_id,bank_to,account_to,amount,k_symbol
0,29401,1,YZ,87144583,2452.0,SIPO
1,29402,2,ST,89597016,3372.7,UVER
2,29403,2,QR,13943797,7266.0,SIPO
3,29404,3,WX,83084338,1135.0,SIPO
4,29405,3,CD,24485939,327.0,Other
5,29406,3,AB,59972357,3539.0,POJISTNE
6,29407,4,UV,26693541,2078.0,SIPO
7,29408,4,UV,5848086,1285.0,SIPO
8,29409,5,GH,37390208,2668.0,SIPO
9,29410,6,AB,44486999,3954.0,SIPO


In [92]:
order_dum = pd.get_dummies(order_df[['k_symbol']])

order_all = pd.concat([order_df,order_dum],axis=1,ignore_index=False)  

order_all = order_all.drop(columns = ['order_id','bank_to','account_to','k_symbol'])

for col in order_dum.columns:
    order_all[col] = order_all[col] * order_all['amount']

order_acc = order_all.groupby('account_id').sum()

order_acc = order_acc.rename(columns={"amount": "amount_order"})

order_acc.head()

,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER
account_id,,,,,,
1,2452.0,0.0,0.0,0.0,2452.0,0.0
2,10638.7,0.0,0.0,0.0,7266.0,3372.7
3,5001.0,0.0,327.0,3539.0,1135.0,0.0
4,3363.0,0.0,0.0,0.0,3363.0,0.0
5,2668.0,0.0,0.0,0.0,2668.0,0.0


In [93]:
account_df.head(10)

,account_id,district_id,frequency,date
0,1,18,POPLATEK MESICNE,1995-03-24
1,2,1,POPLATEK MESICNE,1993-02-26
2,3,5,POPLATEK MESICNE,1997-07-07
3,4,12,POPLATEK MESICNE,1996-02-21
4,5,15,POPLATEK MESICNE,1997-05-30
5,6,51,POPLATEK MESICNE,1994-09-27
6,7,60,POPLATEK MESICNE,1996-11-24
7,8,57,POPLATEK MESICNE,1995-09-21
8,9,70,POPLATEK MESICNE,1993-01-27
9,10,54,POPLATEK MESICNE,1996-08-28


In [94]:
account_df = pd.merge(account_df, order_acc, on='account_id', how='left')

account_df.head()

#Order now done.... onto to transactions

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0


In [95]:
#First half of transactions

trans_1_df = pd.read_csv("trans_1.csv",parse_dates=[2])

trans_1_df = trans_1_df.drop(columns=['trans_id','bank','account'])

trans_1_df['k_symbol'] = trans_1_df['k_symbol'].replace(r'^\s*$', np.nan, regex=True)

trans_1_df['operation'] = trans_1_df['operation'].fillna('Other')
trans_1_df['k_symbol'] = trans_1_df['k_symbol'].fillna('Other')

trans_1_df_dum = pd.get_dummies(trans_1_df[['type','operation','k_symbol']])

trans_1_df_all = pd.concat([trans_1_df,trans_1_df_dum],axis=1,ignore_index=False)  

for col in trans_1_df_dum.columns:
    trans_1_df_all[col] = trans_1_df_all[col] * trans_1_df_all['amount']

    
trans_1_df_all = trans_1_df_all.drop(columns = ['type','operation','k_symbol'])    

trans_1_df_all.head()


,account_id,date,amount,balance,type_PRIJEM,type_VYBER,type_VYDAJ,operation_Other,operation_PREVOD NA UCET,operation_PREVOD Z UCTU,operation_VKLAD,operation_VYBER,operation_VYBER KARTOU,k_symbol_DUCHOD,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SANKC. UROK,k_symbol_SIPO,k_symbol_SLUZBY,k_symbol_UROK,k_symbol_UVER
0,1,1995-04-30,19,17298,19,0,0,19,0,0,0,0,0,0,0,0,0,0,0,19,0
1,1,1995-05-31,79,23156,79,0,0,79,0,0,0,0,0,0,0,0,0,0,0,79,0
2,1,1995-06-30,101,26736,101,0,0,101,0,0,0,0,0,0,0,0,0,0,0,101,0
3,1,1995-07-31,109,25224,109,0,0,109,0,0,0,0,0,0,0,0,0,0,0,109,0
4,1,1995-08-31,99,21502,99,0,0,99,0,0,0,0,0,0,0,0,0,0,0,99,0


In [96]:
trans_acc = trans_1_df_all.groupby('account_id').agg({ 'date' : ['min','max'],
                            'amount' : ['sum','mean','std'],
                            'balance': ['min','max','mean','std'],
                            'type_PRIJEM' : ['sum'],
                            'type_VYBER': ['sum'],
                            'type_VYDAJ': ['sum'], 
                            'operation_Other': ['sum'], 
                            'operation_PREVOD NA UCET': ['sum'],
                            'operation_PREVOD Z UCTU': ['sum'], 
                            'operation_VKLAD': ['sum'],
                            'operation_VYBER': ['sum'],
                            'operation_VYBER KARTOU': ['sum'], 
                            'k_symbol_DUCHOD': ['sum'],
                            'k_symbol_Other': ['sum'], 
                            'k_symbol_POJISTNE': ['sum'], 
                            'k_symbol_SANKC. UROK': ['sum'],
                            'k_symbol_SIPO': ['sum'], 
                            'k_symbol_SLUZBY': ['sum'], 
                            'k_symbol_UROK': ['sum'], 
                            'k_symbol_UVER': ['sum']
                            })



trans_acc.columns = ["_".join(x) for x in trans_acc.columns.ravel()]

trans_acc.head()

,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0
2,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952
3,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0
4,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0
5,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0


In [97]:
#Second Half of Transactions

trans_2_df = pd.read_csv("trans_2.csv",parse_dates=[2])

trans_2_df = trans_2_df.drop(columns=['trans_id','bank','account'])

trans_2_df['k_symbol'] = trans_2_df['k_symbol'].replace(r'^\s*$', np.nan, regex=True)

trans_2_df['operation'] = trans_2_df['operation'].fillna('Other')
trans_2_df['k_symbol'] = trans_2_df['k_symbol'].fillna('Other')

trans_2_df_dum = pd.get_dummies(trans_2_df[['type','operation','k_symbol']])

trans_2_df_all = pd.concat([trans_2_df,trans_2_df_dum],axis=1,ignore_index=False)  

for col in trans_2_df_dum.columns:
    trans_2_df_all[col] = trans_2_df_all[col] * trans_2_df_all['amount']

trans_2_df_all = trans_2_df_all.drop(columns = ['type','operation','k_symbol'])    

trans_2_acc = trans_2_df_all.groupby('account_id').agg({ 'date' : ['min','max'],
                            'amount' : ['sum','mean','std'],
                            'balance': ['min','max','mean','std'],
                            'type_PRIJEM' : ['sum'],
                            'type_VYBER': ['sum'],
                            'type_VYDAJ': ['sum'], 
                            'operation_Other': ['sum'], 
                            'operation_PREVOD NA UCET': ['sum'],
                            'operation_PREVOD Z UCTU': ['sum'], 
                            'operation_VKLAD': ['sum'],
                            'operation_VYBER': ['sum'],
                            'operation_VYBER KARTOU': ['sum'], 
                            'k_symbol_DUCHOD': ['sum'],
                            'k_symbol_Other': ['sum'], 
                            'k_symbol_POJISTNE': ['sum'], 
                            'k_symbol_SANKC. UROK': ['sum'],
                            'k_symbol_SIPO': ['sum'], 
                            'k_symbol_SLUZBY': ['sum'], 
                            'k_symbol_UROK': ['sum'], 
                            'k_symbol_UVER': ['sum']
                            })



trans_2_acc.columns = ["_".join(x) for x in trans_2_acc.columns.ravel()]

trans_2_acc.head()

,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
account_id,,,,,,,,,,,,,,,,,,,,,,,,,,
2303,1996-11-24,1997-06-12,158973,19871.625000,9016.050417,500,71724,48244.125000,23036.203807,158973,0,0,0,0,0,158973,0,0,0,158973,0,0,0,0,0,0
2304,1996-02-19,1998-12-31,1584512,12188.553846,15375.428137,500,113037,40727.107692,18962.373629,805181,106101,673230,5521,0,0,799660,779331,0,0,1578661,0,0,0,330,5521,0
2305,1993-12-11,1998-12-08,519163,1736.331104,1841.551952,-23685,28575,10074.070234,12929.605440,250524,0,268639,2704,0,212520,35300,268639,0,0,424150,0,2284,89500,525,2704,0
2306,1993-12-08,1998-12-31,1701423,4119.668281,5213.399198,400,64467,32824.578692,10015.992308,875658,0,825765,8168,266840,0,867490,558925,0,0,1626406,0,0,66024,825,8168,0
2307,1995-07-12,1998-12-31,420099,1834.493450,2184.091749,600,28375,17853.292576,4306.683369,221501,0,198598,2963,127983,217938,600,70615,0,217938,75211,0,0,123432,555,2963,0


In [98]:
trans_all = pd.concat([trans_acc,trans_2_acc],axis=0,ignore_index=False)  

trans_all = trans_all.groupby(trans_all.index).first()

trans_all = trans_all.reset_index()

trans_all['account_id'] = trans_all['account_id'].astype('int64')

trans_all.head(10)

,account_id,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
0,1,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0
1,2,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952
2,3,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0
3,4,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0
4,5,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0
5,6,1994-09-27,1998-12-31,647584,2632.455285,2655.001376,900,51879,34547.353659,7331.711741,347616,0,299968,6597,185838,340119,900,114130,0,340119,114340,0,0,185838,690,6597,0
6,7,1996-11-24,1998-12-31,1192047,9169.592308,10149.268190,900,99675,62598.600000,14932.471892,630267,0,561780,6492,97600,0,623775,458280,5900,0,1087655,0,0,97600,300,6492,0
7,8,1995-09-21,1998-12-31,1712923,6743.791339,8144.286604,900,79255,46819.429134,13128.157343,892765,0,820158,7681,319628,0,885084,500530,0,0,1613312,0,0,91420,510,7681,0
8,9,1993-01-27,1998-12-31,696458,1929.246537,4993.030943,400,85444,26625.307479,11555.388740,360153,0,336305,7596,0,0,352557,336305,0,0,687857,0,0,0,1005,7596,0
9,10,1996-08-28,1998-12-31,1040615,6670.608974,7335.994095,1100,83410,47792.211538,12152.299822,549951,0,490664,5528,161759,0,544423,328905,0,0,872983,0,0,161759,345,5528,0


In [99]:
account_df = pd.merge(account_df, trans_all, on='account_id', how='left')

account_df.head()

#Transactions added

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0


In [100]:
dist_df = pd.read_csv("district.csv")

dist_df = dist_df.drop(columns = ['A2','A3']) 

dist_df.head()



,district_id,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
1,2,88884,80,26,6,2,5,46.7,8507,1.6,1.85,132,2159.0,2674
2,3,75232,55,26,4,1,5,41.7,8980,1.9,2.21,111,2824.0,2813
3,4,149893,63,29,6,2,6,67.4,9753,4.6,5.05,109,5244.0,5892
4,5,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040


In [101]:
account_df = pd.merge(account_df, dist_df, on='district_id', how='left')

account_df.head()

#District Complete, now merge card and client onto disp

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0,107870,84,29,6,1,6,58.0,8754,3.8,4.31,137,3804.0,3868
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0,58796,22,16,7,1,5,51.9,9045,3.1,3.60,124,1845.0,1879


In [102]:
disp_df = pd.read_csv("disp.csv")
client_df = pd.read_csv("client.csv")
card_df = pd.read_csv("card.csv",parse_dates=[3])

card_df.head()

,card_id,disp_id,type,issued
0,1,9,gold,1998-10-16
1,2,19,classic,1998-03-13
2,3,41,gold,1995-09-03
3,4,42,classic,1998-11-26
4,5,51,junior,1995-04-24


In [103]:
disp_df = pd.merge(disp_df, client_df, on='client_id',how='left')
disp_df = pd.merge(disp_df, card_df, on='disp_id',how='left')

disp_df_all = disp_df.groupby('account_id').first()

disp_df_all = disp_df_all.drop(columns = ['disp_id','client_id','district_id','card_id']) 

disp_df_all = disp_df_all.reset_index()

disp_df_all['account_id'] = disp_df_all['account_id'].astype('int64')

disp_df_all['type_y'] = disp_df_all['type_y'].fillna('No Card')

disp_df_all.head()

,account_id,type_x,gender,birth_date,type_y,issued
0,1,OWNER,F,1970-12-13,No Card,NaT
1,2,OWNER,M,1945-02-04,No Card,NaT
2,3,OWNER,M,1956-12-01,No Card,NaT
3,4,OWNER,M,1919-09-22,No Card,NaT
4,5,OWNER,M,1929-01-25,No Card,NaT


In [104]:
account_df = pd.merge(account_df, disp_df_all, on='account_id',how='left')


#account_df.to_csv('accounts_merged_all.csv',index=False)


account_df.head()

,account_id,district_id,frequency,date,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,date_min,date_max,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,type_x,gender,birth_date,type_y,issued
0,1,18,POPLATEK MESICNE,1995-03-24,2452.0,0.0,0.0,0.0,2452.0,0.0,1995-03-24,1998-12-31,375192,1569.841004,1732.115246,1000,30415,15994.079498,3656.858500,194322,0,180870,3067,98080,165555,25700,82790,0,0,273445,0,0,98080,600,3067,0,70699,60,13,2,1,4,65.3,8968,2.8,3.35,131,1740.0,1910,OWNER,F,1970-12-13,No Card,NaT
1,2,1,POPLATEK MESICNE,1993-02-26,10638.7,0.0,0.0,0.0,7266.0,3372.7,1993-02-26,1998-12-31,3151514,6593.125523,7694.515237,1100,69302,36540.780335,11028.155962,1597055,217442,1337017,10719,553242,1537936,48400,1001217,0,0,2586578,0,0,472290,975,10719,80952,1204953,0,0,0,1,1,100.0,12541,0.2,0.43,167,85677.0,99107,OWNER,M,1945-02-04,No Card,NaT
2,3,5,POPLATEK MESICNE,1997-07-07,5001.0,0.0,327.0,3539.0,1135.0,0.0,1997-07-07,1998-12-31,295030,2521.623932,2951.300633,1000,53447,26973.529915,10508.728242,173062,0,121968,2073,65013,0,170989,56955,0,0,232000,46007,0,14755,195,2073,0,95616,65,30,4,1,6,51.4,9307,3.8,4.43,118,2616.0,3040,OWNER,M,1956-12-01,No Card,NaT
3,4,12,POPLATEK MESICNE,1996-02-21,3363.0,0.0,0.0,0.0,3363.0,0.0,1996-02-21,1998-12-31,350986,1887.021505,1956.829637,800,34870,21753.526882,5009.976472,192349,0,158637,2747,98812,188802,800,59825,0,188802,98740,0,0,60262,435,2747,0,107870,84,29,6,1,6,58.0,8754,3.8,4.31,137,3804.0,3868,OWNER,M,1919-09-22,No Card,NaT
4,5,15,POPLATEK MESICNE,1997-05-30,2668.0,0.0,0.0,0.0,2668.0,0.0,1997-05-30,1998-12-31,166891,1986.797619,1981.162764,600,32036,25090.023810,4752.452095,97489,0,69402,1566,37352,95323,600,32050,0,95323,32440,0,0,37352,210,1566,0,58796,22,16,7,1,5,51.9,9045,3.1,3.60,124,1845.0,1879,OWNER,M,1929-01-25,No Card,NaT


In [105]:
account_df.shape

(4500, 54)

In [111]:
#Merge onto our loans

loans_df = pd.read_csv("loan.csv",parse_dates=[2])

loans_base_df = pd.merge(loans_df, account_df, on='account_id',how='left')



print(loans_base_df.head(10))

loans_base_df.to_csv('loan_merged_original.csv')


   loan_id  account_id     date_x  amount  duration  payments status  \
0     4959           2 1994-01-05   80952        24    3373.0      A   
1     4961          19 1996-04-29   30276        12    2523.0      B   
2     4962          25 1997-12-08   30276        12    2523.0      A   
3     4967          37 1998-10-14  318480        60    5308.0      D   
4     4968          38 1998-04-19  110736        48    2307.0      C   
5     4973          67 1996-05-02  165960        24    6915.0      A   
6     4986          97 1997-08-10  102876        12    8573.0      A   
7     4988         103 1997-12-06  265320        36    7370.0      D   
8     4989         105 1998-12-05  352704        48    7348.0      C   
9     4990         110 1997-09-08  162576        36    4516.0      C   

   district_id         frequency      date_y  amount_order  k_symbol_LEASING  \
0            1  POPLATEK MESICNE  1993-02-26       10638.7               0.0   
1           21  POPLATEK MESICNE  1995-04-07   

In [107]:
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

num_var = [ 'amount_order','k_symbol_LEASING',
            'k_symbol_Other', 'k_symbol_POJISTNE',
            'k_symbol_SIPO', 'k_symbol_UVER',
            'amount_sum',
            'amount_mean', 'amount_std', 
            'balance_min', 'balance_max', 'balance_mean', 'balance_std',
            'type_PRIJEM_sum', 'type_VYBER_sum',
            'type_VYDAJ_sum', 'operation_Other_sum', 'operation_PREVOD NA UCET_sum',
            'operation_PREVOD Z UCTU_sum', 'operation_VKLAD_sum',
            'operation_VYBER_sum', 'operation_VYBER KARTOU_sum',
            'k_symbol_DUCHOD_sum', 
            'k_symbol_Other_sum', 'k_symbol_POJISTNE_sum',
            'k_symbol_SANKC. UROK_sum', 'k_symbol_SIPO_sum', 'k_symbol_SLUZBY_sum',
            'k_symbol_UROK_sum', 
            'k_symbol_UVER_sum', 
            'A4', 'A5', 'A6', 'A7', 'A8',
            'A9', 'A10', 'A11', 'A12', 'A13', 'A14', 'A15', 'A16']

ord_var = ['gender','frequency','type_y']
date_var = ['date_x','date_y', 'date_min', 'date_max','birth_date']
drop_var = ['amount_x','duration','payments','loan_id','account_id','district_id','type_x','issued']
#pass_col = 

loans_base_df = loans_base_df.replace([np.inf, -np.inf], np.nan)

target_raw = loans_base_df['status']
target_raw[(target_raw == 'A') | (target_raw == 'C')] = 0
target_raw[(target_raw == 'B')  | (target_raw == 'D')] = 1
#Build a column transforrner

numeric_transformer = SimpleImputer(missing_values = np.nan, strategy='median')
  
binary_transformer = OneHotEncoder(handle_unknown='ignore')

ordinal_transformer = OneHotEncoder(categories  = [ ['M','F'],
                                                     ['POPLATEK PO OBRATU','POPLATEK TYDNE','POPLATEK MESICNE'],
                                                     ['No Card','junior','classic','gold']])

preprocessor = ColumnTransformer(transformers = [ ('numerical', numeric_transformer, num_var),
                                                  ('ordinal',ordinal_transformer, ord_var)],
                                                  #('date','passthrough', date_var)],
                                                   remainder = 'drop')

X = loans_base_df.drop(columns = 'status')
y = target_raw 


#Save X, Y

#loans_base_df.to_csv('loan_merged_original.csv')


X_train, X_val, y_train, y_val = train_test_split(X,y,test_size=0.2, random_state = 42)


/home/dan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/dan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [108]:
print(loans_base_df['status'])
loans_base_df.to_csv('loan_merged_orignal.csv')

0      0
1      1
2      0
3      1
4      0
      ..
677    0
678    0
679    0
680    0
681    0
Name: status, Length: 682, dtype: object


In [78]:
X_train_transformed = preprocessor.fit_transform(X_train)

X_val_transformed = preprocessor.fit_transform(X_val)

y_val = y_val.astype(bool)


In [79]:

ord_var_list = ['M','F'] +  ['POPLATEK PO OBRATU','POPLATEK TYDNE','POPLATEK MESICNE'] + ['No Card','junior','classic','gold']

transformed_columns = num_var +  ord_var_list # + date_var

print(X_train_transformed.shape)
print(len(transformed_columns))

X_train_transform_df = pd.DataFrame(X_train_transformed, columns = transformed_columns)

X_train_transform_df.head()

(545, 52)
52


,amount_order,k_symbol_LEASING,k_symbol_Other,k_symbol_POJISTNE,k_symbol_SIPO,k_symbol_UVER,amount_sum,amount_mean,amount_std,balance_min,balance_max,balance_mean,balance_std,type_PRIJEM_sum,type_VYBER_sum,type_VYDAJ_sum,operation_Other_sum,operation_PREVOD NA UCET_sum,operation_PREVOD Z UCTU_sum,operation_VKLAD_sum,operation_VYBER_sum,operation_VYBER KARTOU_sum,k_symbol_DUCHOD_sum,k_symbol_Other_sum,k_symbol_POJISTNE_sum,k_symbol_SANKC. UROK_sum,k_symbol_SIPO_sum,k_symbol_SLUZBY_sum,k_symbol_UROK_sum,k_symbol_UVER_sum,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16,M,F,POPLATEK PO OBRATU,POPLATEK TYDNE,POPLATEK MESICNE,No Card,junior,classic,gold
0,6678.7,0.0,0.0,0.0,0.0,6678.7,3976149.0,14049.996466,16293.266594,500.0,127567.0,63984.127208,20995.372905,2016160.0,155325.0,1804664.0,12060.0,207049.0,2003600.0,500.0,1727340.0,25600.0,0.0,3756425.0,0.0,0.0,0.0,615.0,12060.0,207049.0,145688.0,22.0,41.0,13.0,2.0,5.0,40.7,8544.0,1.2,1.86,109.0,2719.0,2906.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,7843.0,0.0,912.0,3103.0,3195.0,633.0,1539421.0,3791.677340,5105.493284,500.0,48157.0,27463.756158,7269.835797,782159.0,0.0,757262.0,5262.0,339642.0,0.0,776897.0,417620.0,0.0,0.0,1234897.0,139635.0,0.0,143775.0,660.0,5262.0,15192.0,122603.0,25.0,21.0,6.0,2.0,8.0,80.0,8991.0,1.3,2.01,128.0,5198.0,5273.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,3160.7,0.0,0.0,0.0,0.0,3160.7,4306964.0,10353.278846,13694.289980,-670.0,138621.0,47080.875000,27197.536554,2208028.0,479514.0,1619422.0,16992.0,142245.0,0.0,2191036.0,1956691.0,0.0,0.0,4146950.0,0.0,12.0,0.0,765.0,16992.0,142245.0,323870.0,0.0,0.0,0.0,1.0,1.0,100.0,10673.0,4.7,5.44,100.0,18782.0,18347.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
3,16476.0,0.0,0.0,0.0,13691.0,2785.0,2804883.0,12577.950673,14677.435013,700.0,109016.0,39265.506726,17914.326139,1415124.0,104538.0,1285221.0,4735.0,480121.0,1402089.0,8300.0,909638.0,0.0,0.0,2317027.0,0.0,0.0,424421.0,3000.0,4735.0,55700.0,58796.0,22.0,16.0,7.0,1.0,5.0,51.9,9045.0,3.1,3.60,124.0,1845.0,1879.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
4,7485.0,0.0,4235.0,0.0,1978.0,1272.0,2173132.0,4734.492375,6214.145313,900.0,57224.0,33080.690632,9437.582439,1102588.0,0.0,1070544.0,8596.0,408984.0,0.0,1093992.0,661560.0,0.0,0.0,1991872.0,0.0,0.0,110768.0,840.0,8596.0,61056.0,1204953.0,0.0,0.0,0.0,1.0,1.0,100.0,12541.0,0.2,0.43,167.0,85677.0,99107.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0


In [80]:
X_train_transform_df.isna().sum()

amount_order                    0
k_symbol_LEASING                0
k_symbol_Other                  0
k_symbol_POJISTNE               0
k_symbol_SIPO                   0
k_symbol_UVER                   0
amount_sum                      0
amount_mean                     0
amount_std                      0
balance_min                     0
balance_max                     0
balance_mean                    0
balance_std                     0
type_PRIJEM_sum                 0
type_VYBER_sum                  0
type_VYDAJ_sum                  0
operation_Other_sum             0
operation_PREVOD NA UCET_sum    0
operation_PREVOD Z UCTU_sum     0
operation_VKLAD_sum             0
operation_VYBER_sum             0
operation_VYBER KARTOU_sum      0
k_symbol_DUCHOD_sum             0
k_symbol_Other_sum              0
k_symbol_POJISTNE_sum           0
k_symbol_SANKC. UROK_sum        0
k_symbol_SIPO_sum               0
k_symbol_SLUZBY_sum             0
k_symbol_UROK_sum               0
k_symbol_UVER_

In [81]:
from sklearn.ensemble import RandomForestClassifier
import eli5


rf_model = RandomForestClassifier(oob_score= True, random_state = 42)

rf_model.fit(X_train_transform_df, y_train.astype(bool))

eli5.show_weights(rf_model, feature_names = transformed_columns)

/home/dan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/dan/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weight,Feature
0.3779 ± 0.7071,balance_min
0.3151 ± 0.6146,k_symbol_SANKC. UROK_sum
0.0385 ± 0.1213,balance_mean
0.0243 ± 0.0798,balance_std
0.0213 ± 0.0765,k_symbol_SIPO
0.0183 ± 0.0708,k_symbol_UVER
0.0129 ± 0.0487,operation_Other_sum
0.0119 ± 0.0431,balance_max
0.0117 ± 0.0575,operation_VKLAD_sum
0.0109 ± 0.0390,k_symbol_UROK_sum


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
     'ccp_alpha': [ 0.0, 0.01 ],
     'class_weight' : [ 'balanced'],
     'max_depth': [ 5, 10, 15, None],
     'max_features': ['auto'],
     'max_leaf_nodes': [ 10, 12, 14 , None],
     'min_impurity_decrease': [ 0.0, 0.001 ],
     'min_samples_leaf': [ 2, 4, 6],
     'min_samples_split': [ 2, 3, 4, 5 ],
     'min_weight_fraction_leaf': [ 0.0, 0.01 ],
     'n_estimators': [50, 65, 80, 95, 110, 150]
}

grid_search = RandomizedSearchCV(rf_model, param_grid, cv=5, verbose=1, random_state=42, n_iter=300, scoring='balanced_accuracy')
grid_search.fit(X_train_transform_df, y_train.astype(bool))

print(("best RF from grid search: %.3f"
       % grid_search.score(X_val_transformed, y_val)))

Fitting 5 folds for each of 300 candidates, totalling 1500 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


In [ ]:
results  = pd.DataFrame(grid_search.cv_results_)

y_val.sum()

In [ ]:
results.sort_values(by='rank_test_score')

In [ ]:
y_pred_val = grid_search.predict(X_val_transformed)


from sklearn.metrics import confusion_matrix

x = confusion_matrix(y_val, y_pred_val)

tn = x[0,0]  
fp = x[0,1] 
fn = x[1,0] 
tp = x[1,1]

print([tp,fp])
print([fn,tn])



In [ ]:
eli5.show_weights(grid_search.best_estimator_, feature_names = transformed_columns)

In [ ]:
#Apply Model to other accounts, then see how they cluster...

grid_search.best_params_

In [ ]:
account_df = pd.read_csv("accounts_merged_all.csv")

account_transformed = preprocessor.fit_transform(account_df)

account_transform_df = pd.DataFrame(account_transformed, columns = transformed_columns)

In [ ]:
account_transform_df['account_id'] = account_df['account_id']

In [ ]:
account_transform_df.shape

In [ ]:
loans_base_df.shape

In [ ]:
(~account_transform_df['account_id'].isin(loans_base_df['account_id'])).sum()

In [ ]:
account_df.isna().sum()

In [ ]:
account_no_loan_df = account_transform_df[(~account_transform_df['account_id'].isin(loans_base_df['account_id']))]

account_no_loan_df = account_no_loan_df.drop(columns='account_id')

account_no_loan_df.head()

In [ ]:
y_acc_pred_val = grid_search.best_estimator_.predict(account_no_loan_df)


In [ ]:
no_loans_df = account_no_loan_df[y_acc_pred_val==1]

preapp_loans_df = account_no_loan_df[y_acc_pred_val==0]

print(no_loans_df.shape)
print(preapp_loans_df.shape)


In [ ]:
X_zero = no_loans_df
X_one = preapp_loans_df

A = (X_one.describe().T - X_zero.describe().T)

print(A.sort_values(by ='mean', ascending=False))

In [ ]:
# Do Some SHAP value!?!?!?!

import shap

rf_explainer = shap.TreeExplainer(grid_search.best_estimator_)
rf_shap_values = rf_explainer.shap_values(X_val_transformed)

shap.initjs()
shap.force_plot(rf_explainer.expected_value[1],
                rf_shap_values[1],
                feature_names= transformed_columns)

shap.initjs()
shap.summary_plot(rf_shap_values[1],
                  features = X_val_transformed, 
                  feature_names = transformed_columns)